In [2]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
import datetime
import py7zr

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]


startDate = "0418"
endDate = "0508"
year = '2020'

readPath = 'H:\\2020 data\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

bad = []
diff = {}
for col in ['Date', 'Data', 'Condition', 'Prob', 'Time']:
    diff[col] = []
    
    
for p in dataPathLs:
    print("*************************************************************************************************")
    if len(np.array(glob.glob(p +'\\***'))) == 0:
        continue
        
    # am & pm
    else:
        os.chdir(p)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(p + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(p + '\\am_hq_snap_spot.7z')
            bad.append(p + '\\am_hq_snap_spot.7z')
            continue
        a.extractall(path = p)
        a.close()
        os.system("copy /b am_snap_level_spot.7z.* am_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(p + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(p + '\\am_snap_level_spot.7z')
            bad.append(p + '\\am_snap_level_spot.7z')
            continue
        a.extractall(path = p)
        a.close()
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(p + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(p + '\\pm_hq_snap_spot.7z')
            bad.append(p + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = p)
        a.close()
        os.system("copy /b pm_snap_level_spot.7z.* pm_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(p + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(p + '\\pm_snap_level_spot.7z')
            bad.append(p + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = p)
        a.close()

        
        am_snap1 = pd.read_table(p + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1
        am_snap2 = pd.read_table(p + "\\am_snap_level_spot.txt",header=None)
        assert(am_snap2.shape[1] == len(columns2))
        am_snap2.columns = columns2
        dd1 = int(am_snap2["OrigTime"].iloc[0]//1000000000*1000000000) + 93000000
        dd2 = int(am_snap2["OrigTime"].iloc[0]//1000000000*1000000000) + 150000000
        am = pd.merge(am_snap2, am_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"], how="outer")
        try:
            assert((am.shape[0] == am_snap2.shape[0]) & (am.shape[0] == am_snap1.shape[0]) & (am_snap1.shape[0] == am_snap2.shape[0]))
        except:
            if am.shape[0] == am_snap1.shape[0]:
                print("am_snap1 have more ticks than am_snap2")
                if all(am[(am["dbtime_x"].isnull()) & ((am["SecurityID"] < 4000) | (am["SecurityID"] > 200000))]["OrigTime"].unique() < dd1):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens after 9:30")      
            if am.shape[0] == am_snap2.shape[0]:
                print("am_snap2 have more ticks than am_snap1")
                if all(am[(am["dbtime_y"].isnull()) & ((am["SecurityID"] < 4000) | (am["SecurityID"] > 200000))]["OrigTime"].unique() < dd1):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens after 9:30")
            else:
                print("am_snap2 don't join with am_snap1")
            am = pd.merge(am_snap2, am_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"])       
        del am_snap1
        del am_snap2
              
        pm_snap1 = pd.read_table(p + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        pm_snap2 = pd.read_table(p + "\\pm_snap_level_spot.txt",header=None)
        assert(pm_snap2.shape[1] == len(columns2))
        pm_snap2.columns = columns2
        pm = pd.merge(pm_snap2, pm_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"], how="outer")
        try:
            assert((pm.shape[0] == pm_snap2.shape[0]) & (pm.shape[0] == pm_snap1.shape[0]) & (pm_snap1.shape[0] == pm_snap2.shape[0]))
        except:
            if pm.shape[0] == pm_snap1.shape[0]:
                print("pm_snap1 have more ticks than pm_snap2")
                if all(pm[(pm["dbtime_x"].isnull()) & ((pm["SecurityID"] < 4000) | (pm["SecurityID"] > 200000))]["OrigTime"].unique() > dd2):
                    print("More ticks happens after 15:00")
                else:
                    print("There are ticks happens before 15:00")
            if pm.shape[0] == pm_snap2.shape[0]:
                print("pm_snap2 have more ticks than pm_snap1")
                if all(pm[(pm["dbtime_y"].isnull()) & ((pm["SecurityID"] < 4000) | (pm["SecurityID"] > 200000))]["OrigTime"].unique() > dd2):
                    print("More ticks happens after 15:00")
                else:
                    print("There are ticks happens before 15:00")
            else:
                print("pm_snap2 don't join with pm_snap1")
            pm = pd.merge(pm_snap2, pm_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"])   
        del pm_snap1
        del pm_snap2
        snapshot = pd.concat([am, pm]).sort_values(by=["SecurityID", "OrigTime"])
        del am
        del pm
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 200000)]
        
        print("am & pm data")
    
    
    path = "\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_" + "2020" + os.path.basename(p).split('.')[0] + "_zs_92_01_day_data\\mdLog_SZ_***" 
    if len(np.array(glob.glob(path))) == 0:
        print(os.path.basename(p).split('.')[0])
        print("92 SZE snapshot data is missing")
        continue
    if len(np.array(glob.glob(path))) == 2:
        logSZ11 = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSZ22 = pd.read_csv(np.array(glob.glob(path))[1], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSZ = pd.concat([logSZ11, logSZ22])
        del logSZ11
        del logSZ22
    else:
        logSZ = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    
    
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    sl = logSZ["StockID"].unique()
    logSZ1 = snapshot
    logSZ1 = logSZ1[logSZ1["StockID"].isin(sl)]
    dd = int(logSZ1["OrigTime"].iloc[0]//1000000000 * 1000000000)
    logSZ1['time'] = (logSZ1['OrigTime'] - dd).astype(int)
    
    # lv2 data comparison
    display(year + os.path.basename(p).split('.')[0] + " SZ lv2 check:")
    data1 = logSZ[(logSZ["time"] > 92500000) & (logSZ["time"] < 145700000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] > 92500000) & (logSZ1["time"] < 145700000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    del data1
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    del data2
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    if n1 > n2:
        data1_1 = data1_1[data1_1["StockID"].isin(data2_1["StockID"].unique())]
    if n2 > n1:
        data2_1 = data2_1[data2_1["StockID"].isin(data1_1["StockID"].unique())]
    
    data1_1["cum_amount"] = data1_1["cum_amount"].round(2)
    data2_1["cum_amount"] = data2_1["cum_amount"].round(2)
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    diff["Date"].append(os.path.basename(p).split('.')[0])
    diff["Data"].append("lv2 data")
    re = test[(~np.isnan(test["time_y"]))&(~np.isnan(test["time_x"]))]
    diff["Time"].append(re[re["time_x"] == re["time_y"]].shape[0]/re.shape[0])
    if n2 < len1:
        display("SZE lv2 test is not complete:")
        display(test[np.isnan(test["time_y"])])
        print(len(test[np.isnan(test["time_y"])])/n1)
        print(test[np.isnan(test["time_y"])]["time_x"].unique())
        print(test[np.isnan(test["time_y"])]["StockID"].unique())
        diff["Condition"].append("SZE lv2 test is not complete")
        diff["Prob"].append(len(test[np.isnan(test["time_y"])])/n1)
    if len1 == n2:
        display("SZE lv2 test is complete")
        diff["Condition"].append("SZE lv2 test is complete")
        diff["Prob"].append('C')
    
    
#     # SZ trade data comparison
#     display(year + os.path.basename(p) + " SZ trade check:")
#     am_trade = pd.read_table(path3[i],header=None)
#     am_trade.columns = ["Date","OrigTime","SendTime","recvtime","dbtime","ChannelNo","MDStreamID","ApplSeqNum", "SecurityID","SecurityIDSource", "BidApplSeqNum",
#                    "OfferApplSeqNum","TradePrice","TradeQty","ExecType","TransactTime"]
#     pm_trade = pd.read_table(path4[i],header=None)
#     pm_trade.columns = ["Date","OrigTime","SendTime","recvtime","dbtime","ChannelNo","MDStreamID","ApplSeqNum", "SecurityID","SecurityIDSource", "BidApplSeqNum", 
#                         "OfferApplSeqNum","TradePrice","TradeQty","ExecType","TransactTime"]
#     # ---------------------------load 92 data here----------------------------------------------------
#     TradeLog = pd.read_csv(r'D:\work\project 13 lv2 data check\logs_20190315_zs_92_01_day_data\mdTradeLog_20190315_0857.csv',
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
#                                                  "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
#                                                  "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
#                                                  "OfferApplSeqNum"]]
#     TradeLogSZ = TradeLog[TradeLog["exchId"] != 1]
#     del TradeLog
#     sl = TradeLogSZ["SecurityID"].unique()
#     am_trade = am_trade[am_trade["SecurityID"].isin(sl)]
#     pm_trade = pm_trade[pm_trade["SecurityID"].isin(sl)]
#     TradeLogSZ1 = pd.concat([am_trade, pm_trade])[["ApplSeqNum", "SecurityID","SecurityIDSource", "BidApplSeqNum","OrigTime","SendTime","recvtime","dbtime",\
#                    "OfferApplSeqNum","TradePrice","TradeQty","ExecType","TransactTime"]]
#     del am_trade
#     del pm_trade
#     TradeLogSZ1["TransactTime"] = TradeLogSZ1["TransactTime"] - 20190315000000000 
#     TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
#     TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

#     columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeQty","BidApplSeqNum","OfferApplSeqNum"]
#     n1 = len(TradeLogSZ["SecurityID"].unique())
#     n2 = len(TradeLogSZ1["SecurityID"].unique())
#     if n1 > n2:
#         TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(TradeLogSZ1["SecurityID"].unique())]
#     if n2 > n1:
#         TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(TradeLogSZ["SecurityID"].unique())]
#     re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
#     n1 = re["sequenceNo"].count()
#     n2 = re["OrigTime"].count()
#     len1 = len(re)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("SZE trade test is not complete:")
#         display(re[np.isnan(re["OrigTime"])])
#         print(len(re[np.isnan(re["OrigTime"])]))
#         print(np.sort(re[np.isnan(re["OrigTime"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["OrigTime"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["OrigTime"])]["SecurityID"].unique())
#     if (len1 == n2) & (n1 < len1):
#         display("SZE trade baseline is not complete:")
#         display(re[np.isnan(re["sequenceNo"])])
#         print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
#         display(n2-n1)
#     if (len1 == n2) & (len1 == n1):
#         display("SZE trade baseline and test exactly same")
#     del TradeLogSZ
#     del TradeLogSZ1
#     del re
    
#     # SZ order data comparison
#     display(year + os.path.basename(p) + " SZ order check:")
#     am_order = pd.read_table(r"\\192.168.10.30\Kevin_zhenyu\exchange data\LEVEL2_shenzhen\0315\am_hq_order_spot\am_hq_order_spot.txt",header=None)
#     am_order.columns = ["Date","OrigTime","SendTime","recvtime","dbtime","ChannelNo","MDStreamID","ApplSeqNum", "SecurityID","SecurityIDSource", "Price",
#                    "OrderQty","TransactTime","Side","OrderType","ConfirmID","Contactor","ContactInfo","ExpirationDays","ExpirationType"]
#     pm_order = pd.read_table(r"\\192.168.10.30\Kevin_zhenyu\exchange data\LEVEL2_shenzhen\0315\pm_hq_order_spot\pm_hq_order_spot.txt",header=None)
#     pm_order.columns = ["Date","OrigTime","SendTime","recvtime","dbtime","ChannelNo","MDStreamID","ApplSeqNum", "SecurityID","SecurityIDSource", "Price",
#                    "OrderQty","TransactTime","Side","OrderType","ConfirmID","Contactor","ContactInfo","ExpirationDays","ExpirationType"]

#     # ---------------------------load 92 data here----------------------------------------------------
#     OrderLog = pd.read_csv(r'D:\work\project 13 lv2 data check\logs_20190315_zs_92_01_day_data\mdOrderLog_20190315_0857.csv',
#                     encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
#                                                  "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
#                                                  "OrderQty"]]
#     sl = OrderLog["SecurityID"].unique()
#     am_order = am_order[am_order["SecurityID"].isin(sl)]
#     pm_order = pm_order[pm_order["SecurityID"].isin(sl)]
#     OrderLog1 = pd.concat([am_order, pm_order])[["TransactTime", "OrigTime","SendTime","recvtime","dbtime","ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]]
#     del am_order
#     del pm_order
#     OrderLog1["TransactTime"] = OrderLog1["TransactTime"] - 20190315000000000
    
#     OrderLog["OrderType"] = OrderLog["OrderType"].apply(lambda x: str(x))
#     OrderLog1["OrderType"] = OrderLog1["OrderType"].apply(lambda x: str(x))
#     columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'OrderQty', "SecurityID"]
#     n1 = len(OrderLog["SecurityID"].unique())
#     n2 = len(OrderLog1["SecurityID"].unique())
#     if n1 > n2:
#         OrderLog = OrderLog[OrderLog["SecurityID"].isin(OrderLog1["SecurityID"].unique())]
#     if n2 > n1:
#         OrderLog1 = OrderLog1[OrderLog1["SecurityID"].isin(OrderLog["SecurityID"].unique())]
#     ree = pd.merge(OrderLog, OrderLog1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
#     n1 = ree["sequenceNo"].count()
#     n2 = ree["OrigTime"].count()
#     len1 = len(ree)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("SZE order test is not complete:")
#         display(ree[np.isnan(ree["OrigTime"])])
#         print(len(ree[np.isnan(ree["OrigTime"])]))
#         print(np.sort(ree[np.isnan(ree["OrigTime"])]["TransactTime"].unique()))
#         print(len(ree[np.isnan(ree["OrigTime"])]["SecurityID"].unique()))
#         print(ree[np.isnan(ree["OrigTime"])]["SecurityID"].unique())
#     if (len1 == n2) & (n1 < len1):
#         display("SZE order test is complete, baseline is not complete:")
#         display(ree[np.isnan(ree["sequenceNo"])])
#         print(np.sort(ree[np.isnan(ree["sequenceNo"])]["TransactTime"].unique()))
#         print(len(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique()))
#         print(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique())
#         display(n2-n1)
#     if (len1 == n2) & (len1 == n1):
#         display("SZE order baseline and test exactly same")
#     del OrderLog
#     del OrderLog1
#     del ree

diff = pd.DataFrame(diff)
diff.to_csv("F:\\SZ4.csv")
print(diff)
print(bad)

*************************************************************************************************
am & pm data


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'20200420 SZ lv2 check:'

2203
2203
7122614
7122614
7122614
-----------------------------------------------


'SZE lv2 test is complete'

...

*************************************************************************************************
am & pm data


'20200508 SZ lv2 check:'

2217
2217
7202223
7202223
7202223
-----------------------------------------------


'SZE lv2 test is complete'

    Date      Data                 Condition Prob      Time
0   0420  lv2 data  SZE lv2 test is complete    C  1.000000
1   0421  lv2 data  SZE lv2 test is complete    C  1.000000
2   0422  lv2 data  SZE lv2 test is complete    C  1.000000
3   0423  lv2 data  SZE lv2 test is complete    C  1.000000
4   0424  lv2 data  SZE lv2 test is complete    C  1.000000
5   0427  lv2 data  SZE lv2 test is complete    C  1.000000
6   0428  lv2 data  SZE lv2 test is complete    C  1.000000
7   0429  lv2 data  SZE lv2 test is complete    C  1.000000
8   0430  lv2 data  SZE lv2 test is complete    C  1.000000
9   0506  lv2 data  SZE lv2 test is complete    C  0.999888
10  0507  lv2 data  SZE lv2 test is complete    C  1.000000
11  0508  lv2 data  SZE lv2 test is complete    C  1.000000
[]


In [2]:
diff = pd.DataFrame(diff)
diff.to_csv("F:\\SZ3.csv")
print(diff)
print(bad)

    Date      Data                 Condition Prob  Time
0   0309  lv2 data  SZE lv2 test is complete    C   1.0
1   0310  lv2 data  SZE lv2 test is complete    C   1.0
2   0311  lv2 data  SZE lv2 test is complete    C   1.0
3   0312  lv2 data  SZE lv2 test is complete    C   1.0
4   0313  lv2 data  SZE lv2 test is complete    C   1.0
5   0316  lv2 data  SZE lv2 test is complete    C   1.0
6   0317  lv2 data  SZE lv2 test is complete    C   1.0
7   0318  lv2 data  SZE lv2 test is complete    C   1.0
8   0319  lv2 data  SZE lv2 test is complete    C   1.0
9   0320  lv2 data  SZE lv2 test is complete    C   1.0
10  0323  lv2 data  SZE lv2 test is complete    C   1.0
11  0324  lv2 data  SZE lv2 test is complete    C   1.0
12  0325  lv2 data  SZE lv2 test is complete    C   1.0
13  0326  lv2 data  SZE lv2 test is complete    C   1.0
[]


In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
import datetime


year = "2019"
startDate = "0314"
endDate = "0315"

readPath = 'E:\\SZ\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs < endDate)]


diff = {}
for col in ['Date', 'Data', 'Condition', 'Prob', 'Time']:
    diff[col] = []
    
    
for p in dataPathLs:
    print("*************************************************************************************************")
    list1 = np.array(glob.glob(p + '\\***'))
#     if len(list1) < 5:
#         continue
    
    F1 = open(p+"\\Snapshot.pkl", 'rb')
    logSZ1 = pickle.load(F1)
    
    
    if len(os.path.basename(p).split('.')[0]) == 4:
        path = "H:\\recordData\\logs_" + year + os.path.basename(p).split('.')[0] + "_zs_92_01_day_data\\mdLog_SZ_***"  
    else:
        path = "H:\\recordData\\logs_" + year + os.path.basename(p).split('.')[0].split('(')[0] + "_zs_92_01_day_data\\mdLog_SZ_***"  
    if len(np.array(glob.glob(path))) == 0:
        print(os.path.basename(p).split('.')[0])
        print("92 SZE snapshot data is missing")
        continue
    if len(np.array(glob.glob(path))) == 2:
        logSZ11 = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSZ22 = pd.read_csv(np.array(glob.glob(path))[1], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSZ = pd.concat([logSZ11, logSZ22])
        del logSZ11
        del logSZ22
    else:
        logSZ = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    
    
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    sl = logSZ["StockID"].unique()
    logSZ1 = logSZ1[logSZ1["StockID"].isin(sl)]
    dd = int(logSZ1["OrigTime"].iloc[0]//1000000000 * 1000000000)
    logSZ1['time'] = (logSZ1['OrigTime'] - dd).astype(int)
    
    # lv2 data comparison
    display(year + os.path.basename(p).split('.')[0] + " SZ lv2 check:")
    data1 = logSZ[(logSZ["time"] > 92500000) & (logSZ["time"] < 145700000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] > 92500000) & (logSZ1["time"] < 145700000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    del data1
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    del data2
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    data1_1["cum_amount"] = data1_1["cum_amount"].round(2)
    data2_1["cum_amount"] = data2_1["cum_amount"].round(2)
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    diff["Date"].append(os.path.basename(p).split('.')[0])
    diff["Data"].append("lv2 data")
    re = test[(~np.isnan(test["time_y"]))&(~np.isnan(test["time_x"]))]
    diff["Time"].append(re[re["time_x"] == re["time_y"]].shape[0]/re.shape[0])
    if n2 < len1:
        display("SZE lv2 test is not complete:")
        display(test[np.isnan(test["time_y"])])
        print(len(test[np.isnan(test["time_y"])])/n1)
        print(test[np.isnan(test["time_y"])]["time_x"].unique())
        print(test[np.isnan(test["time_y"])]["StockID"].unique())
        diff["Condition"].append("SZE lv2 test is not complete")
        diff["Prob"].append(len(test[np.isnan(test["time_y"])])/n1)
    if len1 == n2:
        display("SZE lv2 test is complete")
        diff["Condition"].append("SZE lv2 test is complete")
        diff["Prob"].append('C')

diff = pd.DataFrame(diff)
# diff.to_csv("F:\\SZ_check_result_2.csv")
print(diff)

*************************************************************************************************


'20190314 SZ lv2 check:'

2133
2133
7776531
7699612
7776531
-----------------------------------------------


'SZE lv2 test is not complete:'

,index_x,clockAtArrival,sequenceNo,source,StockID,exchange,time_x,cum_volume,cum_amount,close,...,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,unknown1,unknown2,unknown3
395386,1059130,1552527608238742,14819211,4,300662,SZ,93948000,247312,9414215.72,38.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395387,1059131,1552527608238754,14819212,4,300667,SZ,93948000,2513900,86681920.50,33.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395388,1059132,1552527608238767,14819213,4,300680,SZ,93948000,108200,2044703.00,18.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395389,1059133,1552527608238773,14819214,4,300683,SZ,93948000,129000,4082509.00,31.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395390,1059134,1552527608238795,14819215,4,300684,SZ,93948000,343900,13478452.60,39.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472400,1216176,1552527728275987,16615602,4,300747,SZ,94148000,302034,51923472.72,171.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472401,1216177,1552527728275995,16615603,4,300750,SZ,94148000,657737,57985464.62,88.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472402,1216178,1552527728276000,16615604,4,300751,SZ,94148000,109500,18225430.00,167.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472403,1216180,1552527728276017,16615606,4,300758,SZ,94148000,731200,37034994.72,50.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0.009891171269040142
[93948000 93951000 93954000 93957000 94000000 94003000 94006000 94009000
 94012000 94015000 94018000 94021000 94024000 94027000 94030000 94033000
 94036000 94039000 94042000 94045000 94048000 94051000 94054000 94057000
 94100000 94103000 94106000 94109000 94112000 94115000 94118000 94121000
 94124000 94127000 94130000 94133000 94136000 94139000 94142000 94145000
 94148000]
[300662 300667 300680 ...    428 300013   2855]
*************************************************************************************************


'20190314(深交所数据) SZ lv2 check:'

2133
2133
7776531
7776531
7776531
-----------------------------------------------


'SZE lv2 test is complete'

          Date      Data                     Condition        Prob     Time
0         0314  lv2 data  SZE lv2 test is not complete  0.00989117  0.99971
1  0314(深交所数据)  lv2 data      SZE lv2 test is complete           C  0.99983


In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
import datetime


year = "2019"
startDate = "0314"
endDate = "0315"

readPath = 'E:\\SZ\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs < endDate)]


diff = {}
for col in ['Date', 'Data', 'Condition', 'Prob', 'Time']:
    diff[col] = []
    
    
for p in dataPathLs[1:]:
    print("*************************************************************************************************")
    list1 = np.array(glob.glob(p + '\\***'))
#     if len(list1) < 5:
#         continue
    
    F1 = open(p+"\\Snapshot.pkl", 'rb')
    logSZ1 = pickle.load(F1)

*************************************************************************************************


In [11]:
logSZ1["ID"] = logSZ1["StockID"].astype(str).apply(lambda x: 'SZ' + x.rjust(6, '0'))

In [2]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SZ' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    db = pd.concat([db, dayData])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
set(db[db["date"] == "2019-03-14"]["ID"].unique()) - set(logSZ1["ID"].unique())

{'SZ001914'}

In [5]:
pd.set_option("max_columns", 200)

In [12]:
display(data1_1[(data1_1["StockID"] == 300059) & (data1_1["cum_volume"] == 221972507)]["cum_amount"].values[0].round(2))
display(data2_1[(data2_1["StockID"] == 300059) & (data2_1["cum_volume"] == 221972507)]["cum_amount"].values[0])

4295120713.15

4295120713.15